<a href="https://colab.research.google.com/github/Beatriz-Yun/AIFFEL_LMS/blob/main/Exploration/%5BE-09%5DMovielens%EC%98%81%ED%99%94%EC%B6%94%EC%B2%9C%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movielens 영화 추천 실습
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 **implicit 데이터로 간주**하고 테스트해볼 수 있습니다.
- 별점을 **시청횟수로 해석**해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 1) 데이터 준비와 전처리
Movielens 데이터는 **rating.dat** 안에 이미 인덱싱까지 완료된 **사용자-영화-평점 데이터**가 깔끔하게 정리되어 있습니다.

In [ ]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)    # 원본데이터 길이

ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)    # 전처리한 데이터 길이

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [ ]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [ ]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [ ]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')

movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2) 데이터 분석
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [ ]:
# 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [ ]:
# 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [ ]:
# 가장 인기 있는 영화 30개
movies_count = ratings.groupby('movie_id')['user_id'].count()

movies_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [ ]:
# 내가 선호하는 영화 5개
my_favorite = ['Little Mermaid, The (1989)' , 'Cinderella (1950)' ,'Sound of Music, The (1965)' ,'Indiana Jones and the Temple of Doom (1984)' ,'Toy Story (1995)']

for i in range(5):
    temp = movies[movies['title'] == my_favorite[i]]
    print(temp)

      movie_id                       title  \
2012      2081  Little Mermaid, The (1989)   

                                            genre  
2012  Animation|Children's|Comedy|Musical|Romance  
      movie_id              title                         genre
1009      1022  Cinderella (1950)  Animation|Children's|Musical
      movie_id                       title    genre
1022      1035  Sound of Music, The (1965)  Musical
      movie_id                                        title             genre
2046      2115  Indiana Jones and the Temple of Doom (1984)  Action|Adventure
   movie_id             title                        genre
0         1  Toy Story (1995)  Animation|Children's|Comedy


In [ ]:
my_favorite = [2081, 1022, 1035, 2115, 1]
my_count = [5.0, 4.0, 5.0, 3.0, 4.0]
my_movielist = pd.DataFrame({'user_id': ['sehui']*5, 'movie_id': my_favorite, 'count': my_count})

if not ratings.isin({'user_id':['sehui']})['user_id'].any():  # user_id에 'sehui'이라는 데이터가 없다면
    ratings = ratings.append(my_movielist)                           # 위에 임의로 만든 my_movielist 데이터를 추가해 줍니다. 

ratings.tail(10)

,user_id,movie_id,count,timestamp
1000203,6040,1090,3.0,956715518.0
1000205,6040,1094,5.0,956704887.0
1000206,6040,562,5.0,956704746.0
1000207,6040,1096,4.0,956715648.0
1000208,6040,1097,4.0,956715569.0
0,sehui,2081,5.0,NaN
1,sehui,1022,4.0,NaN
2,sehui,1035,5.0,NaN
3,sehui,2115,3.0,NaN
4,sehui,1,4.0,NaN


### 추가적인 전처리 (indexing, 칼럼제거)

In [ ]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 영화를 indexing 하는 코드
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [ ]:
user_unique

array([1, 2, 3, ..., 6039, 6040, 'sehui'], dtype=object)

'sehui'라는 user_id를 추가하기 전에 unique한 user_id 수가 6039였으므로 'sehui'의 index는 6039가 맞다. <br>(index는 0부터 시작)

In [ ]:
user_to_idx['sehui']

6039

In [ ]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

temp_user_data = ratings['user_id'].map(user_to_idx).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('artist column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,count,timestamp
0,0,0,5.0,978300760.0
1,0,1,3.0,978302109.0
2,0,2,3.0,978301968.0
3,0,3,4.0,978300275.0
4,0,4,5.0,978824291.0
...,...,...,...,...
0,6039,191,5.0,NaN
1,6039,37,4.0,NaN
2,6039,14,5.0,NaN
3,6039,188,3.0,NaN


In [ ]:
ratings.drop(['timestamp'], axis=1, inplace=True)

In [ ]:
ratings

,user_id,movie_id,count
0,0,0,5.0
1,0,1,3.0
2,0,2,3.0
3,0,3,4.0
4,0,4,5.0
...,...,...,...
0,6039,191,5.0
1,6039,37,4.0
2,6039,14,5.0
3,6039,188,3.0


## 4) CSR matrix를 직접 만들어 봅시다.

In [ ]:
ratings['count']

0    5.0
1    3.0
2    3.0
3    4.0
4    5.0
    ... 
0    5.0
1    4.0
2    5.0
3    3.0
4    4.0
Name: count, Length: 836483, dtype: float64

In [ ]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 설정. 학습내용과 무관.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 \_\_init__ 파라미터
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

파라미터 factors와 iterations를 늘릴수록 학습데이터를 잘 학습하게 되지만 overfitting의 우려가 있으니 좋은 값을 찾아야 한다.

In [ ]:
# Implicit AlternatingLeastSquares 모델의 선언

als_model = AlternatingLeastSquares(factors=640, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float64)

In [ ]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [ ]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [ ]:
sehui, little_mermaid = user_to_idx['sehui'], movie_to_idx[2081]
sehui_vector, little_mermaid_vector = als_model.user_factors[sehui], als_model.item_factors[little_mermaid]

In [ ]:
print(sehui_vector.shape)
print(little_mermaid_vector.shape)

(640,)
(640,)


In [ ]:
np.dot(sehui_vector, little_mermaid_vector)

0.8566565550377852

In [ ]:
toystory_vector = als_model.item_factors[movie_to_idx[1]]    # 영화 toystory
np.dot(sehui_vector, toystory_vector)

0.9208953678670195

In [ ]:
jumanji_vector = als_model.item_factors[movie_to_idx[2]]    # 영화 jumanji
np.dot(sehui_vector, jumanji_vector)

0.017441349059493294

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [ ]:
movie_id = movie_to_idx[2081]    # Little Mermaid, The
similar_movie = als_model.similar_items(movie_id, N=15)

similar_movie

[(191, 1.0000000000000002),
 (3364, 0.3739316296889651),
 (3209, 0.37157465162804304),
 (3607, 0.370803311816906),
 (3498, 0.36944649654995854),
 (3453, 0.3692509743100145),
 (3069, 0.36916949475491423),
 (3550, 0.36906817910525624),
 (3494, 0.3689556696445093),
 (3522, 0.3682760401149685),
 (3609, 0.3669092735599585),
 (3441, 0.36690818895746247),
 (3219, 0.36688771058344605),
 (3215, 0.3667623633626824),
 (3440, 0.36674275108399984)]

In [ ]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[2081,
 1850,
 3883,
 404,
 526,
 1878,
 1817,
 3876,
 790,
 981,
 2486,
 1773,
 3800,
 792,
 3119]

In [ ]:
def get_similar_movie(movie_id: str):
    movie_id = movie_to_idx[movie_id]
    similar_movie = als_model.similar_items(movie_id, N=15)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [ ]:
movie_list = get_similar_movie(1)

movie_list

[1,
 106,
 1324,
 3290,
 1843,
 2909,
 3026,
 1868,
 3305,
 3876,
 690,
 3482,
 776,
 657,
 3899]

In [ ]:
movies.loc[movies['movie_id'].isin(movie_list)]

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
104,106,Nobody Loves Me (Keiner liebt mich) (1994),Comedy|Drama
651,657,Yankee Zulu (1994),Comedy|Drama
681,690,"Promise, The (Versprechen, Das) (1994)",Romance
766,776,Babyfever (1994),Comedy|Drama
1304,1324,Amityville: Dollhouse (1996),Horror
1774,1843,Slappy and the Stinkers (1998),Children's|Comedy
1799,1868,"Truce, The (1996)",Drama|War
2840,2909,"Five Wives, Three Secretaries and Me (1998)",Documentary
2957,3026,Slaughterhouse (1987),Horror


## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [ ]:
user = user_to_idx['sehui']

# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(572, 0.2979205232876652),
 (46, 0.25824278552820373),
 (8, 0.22110959224621005),
 (551, 0.20170217737563442),
 (45, 0.1855378775461952),
 (10, 0.1789025583128658),
 (172, 0.1700095048655152),
 (1844, 0.15861138851572917),
 (360, 0.1549460009995044),
 (547, 0.15421356524194368),
 (536, 0.13955492389554702),
 (548, 0.1234855924748817),
 (330, 0.12184758023047129),
 (860, 0.12140908038572898),
 (119, 0.11695203883660404),
 (326, 0.11662564615214308),
 (1537, 0.11333872459760658),
 (619, 0.11318811761134033),
 (33, 0.11245816055859273),
 (685, 0.11224042103831916)]

In [ ]:
movie_list = [idx_to_movie[i[0]] for i in movie_recommended]

movie_list

[2096,
 1029,
 594,
 2087,
 1028,
 595,
 1291,
 1951,
 1088,
 2080,
 2078,
 1282,
 364,
 1185,
 736,
 1101,
 3555,
 1032,
 588,
 2529]

In [ ]:
movies.loc[movies['movie_id'].isin(movie_list)]

,movie_id,title,genre
360,364,"Lion King, The (1994)",Animation|Children's|Musical
584,588,Aladdin (1992),Animation|Children's|Comedy|Musical
590,594,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
591,595,Beauty and the Beast (1991),Animation|Children's|Musical
727,736,Twister (1996),Action|Adventure|Romance|Thriller
1015,1028,Mary Poppins (1964),Children's|Comedy|Musical
1016,1029,Dumbo (1941),Animation|Children's|Musical
1019,1032,Alice in Wonderland (1951),Animation|Children's|Musical
1072,1088,Dirty Dancing (1987),Musical|Romance
1085,1101,Top Gun (1986),Action|Romance


user데이터가 추천에 기여한 정도 확인

In [ ]:
movie_idx = movie_to_idx[2096]    # 잠자는 숲속의 미녀
explain = als_model.explain(user, csr_data, itemid=movie_idx)

In [ ]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(1022, 0.24156215586026025),
 (2081, 0.02634279082136016),
 (2115, 0.013899383431974058),
 (1, 0.007557615302793256),
 (1035, 0.004722663010809339)]

In [ ]:
movie_list = [idx_to_movie[i[0]] for i in explain[1]]
movies.loc[movies['movie_id'].isin(movie_list)]

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1009,1022,Cinderella (1950),Animation|Children's|Musical
1022,1035,"Sound of Music, The (1965)",Musical
2012,2081,"Little Mermaid, The (1989)",Animation|Children's|Comedy|Musical|Romance
2046,2115,Indiana Jones and the Temple of Doom (1984),Action|Adventure


영화 '신데렐라'가 기여한 정도가 가장 크다.
- 만화영화
- 어린이용
- 뮤지컬영화

등등 공통점이 많기 때문이라고 생각한다.

# 루브릭
1.CSR matrix가 정상적으로 만들어졌다.
- 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.
2.MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.
- 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.
3.비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.
- MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.

# 회고

실습을 하면서 깨달은 건데 처음에 movies 데이터프레임을 ratings 데이터프레임과 movie_id를 기준으로 합친 후에 movie_id칼럼을 제거하고 movie_title에 대하여 indexing을 하는 것이 맞았던 것 같다.

그리고 평점을 영화를 본 횟수로 여겨서 추천시스템을 구성하는 게 색다른 것 같다. 처음엔 의아했지만 생각해보니 평점과 영화를 본 횟수 둘 다 클수록 선호도가 높다고 여길 수도 있기 때문에 합리적인 추천 기준이라고 생각했다. 하지만 이번에 implicit데이터로 간주한 것과 평점으로 진행했을 때의 구현 방법, 성능 등의 차이가 궁금하다.

마지막에 학습한 모델로 유사한 영화를 추천하는데 만화영화에 대한 결과에 무슨 공포영화가 있어서 학습이 잘 안된줄 알았으나, 또 내가 좋아할만한 영화들에 대한 추천은 훨씬 납득가능한 결과여서 신기했고 이러한 결과의 원인도 궁금했다.